Data Cleaning

1. Cleaning: 
  - Conceal Personal Information
  - Handle Missing Values
2. Verify Target Data
3. Separate Storage for Different Data Types (Discrete and Continuous)

### Load original data

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import os

import warnings
warnings.filterwarnings('ignore')

In [19]:
# get the path of the current directory
path = os.getcwd()
path
# get the csv file path
csv_path = os.path.join(path, 'data/loan_data_2007_2014.csv')
csv_path
# create the the folder for the data cleaning
if not os.path.exists('data/1-data_cleaning'):
    os.makedirs('data/1-data_cleaning')
    # save this path to the variable

# save this path: data_cleaning_path
data_cleaning_path = os.path.join(path, 'data/1-data_cleaning')
data_cleaning_path

'/Users/gan-m2/Credit Risk Modelling pipeline/data/1-data_cleaning'

In [8]:
# csv_path = '/Users/gan-m2/1-DataProcessing/data/loan_data_2007_2014.csv'
df = pd.read_csv(csv_path, low_memory=False, index_col=0)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 1. Cleaning

In [9]:
def clean_data(df, personal_info_columns, missing_rate_threshold=0.3):
    """
    Clean the DataFrame by dropping personal information columns and columns with high missing rates.

    Args:
    - df (DataFrame): The input DataFrame to be cleaned.
    - personal_info_columns (list of str): List of column names that contain personal information.
    - missing_rate_threshold (float): Threshold for determining high missing rate columns.

    Returns:
    - DataFrame: The cleaned DataFrame.
    """
    # Calculate the missing rate for each column
    missing_rate = df.isnull().sum() / len(df)

    # Identify columns with a missing rate over the specified threshold
    columns_high_missing_rate = missing_rate[missing_rate > missing_rate_threshold].index.tolist()

    # Drop personal info columns and columns with high missing rate
    columns_to_drop = set(personal_info_columns + columns_high_missing_rate)
    df_cleaned = df.drop(columns=columns_to_drop)

    return df_cleaned

# Example usage
personal_info_columns = ['id', 'member_id', 'emp_title', 'issue_d', 'url', 'desc', 'title', 'zip_code']
df_cleaned = clean_data(df, personal_info_columns)

# Display the first few rows of the cleaned dataframe
df_cleaned.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,5000,5000,4975.0,36 months,10.65,162.87,B,B2,10+ years,RENT,...,Jan-15,171.62,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
1,2500,2500,2500.0,60 months,15.27,59.83,C,C4,< 1 year,RENT,...,Apr-13,119.66,Sep-13,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
2,2400,2400,2400.0,36 months,15.96,84.33,C,C5,10+ years,RENT,...,Jun-14,649.91,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
3,10000,10000,10000.0,36 months,13.49,339.31,C,C1,10+ years,RENT,...,Jan-15,357.48,Jan-15,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
4,3000,3000,3000.0,60 months,12.69,67.79,B,B5,1 year,RENT,...,Jan-16,67.79,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN


In [20]:
# save the cleaned data, use the path i save 
df_cleaned.to_csv(os.path.join(data_cleaning_path, 'loan_data_2007_2014_cleaned.csv'))

# 2. Transform target value into binary

In [21]:
def transform_loan_status(df, bad_loan_statuses):
    """
    Transform the 'loan_status' column into a binary target variable.

    Args:
    - df (DataFrame): The input DataFrame.
    - bad_loan_statuses (list of str): List of statuses considered as 'bad' loans.

    Returns:
    - DataFrame: The transformed DataFrame with a new 'target' column and without the original 'loan_status' column.
    """
    # Transform the loan_status column into a binary target variable
    df['target'] = df['loan_status'].apply(lambda x: 1 if x in bad_loan_statuses else 0)

    # Drop the original loan_status column
    df = df.drop(columns='loan_status')

    return df

# Example usage
bad_loan_statuses = [
    'Charged Off', 
    'Default', 
    'Late (31-120 days)', 
    'Does not meet the credit policy. Status:Charged Off'
]
df_cleaned_transformed = transform_loan_status(df_cleaned, bad_loan_statuses)

# Display the first few rows of the transformed dataframe
df_cleaned_transformed.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,target
0,5000,5000,4975.0,36 months,10.65,162.87,B,B2,10+ years,RENT,...,171.62,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN,0
1,2500,2500,2500.0,60 months,15.27,59.83,C,C4,< 1 year,RENT,...,119.66,Sep-13,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN,1
2,2400,2400,2400.0,36 months,15.96,84.33,C,C5,10+ years,RENT,...,649.91,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN,0
3,10000,10000,10000.0,36 months,13.49,339.31,C,C1,10+ years,RENT,...,357.48,Jan-15,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN,0
4,3000,3000,3000.0,60 months,12.69,67.79,B,B5,1 year,RENT,...,67.79,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN,0


In [22]:
# save the cleaned data, use the path i save
df_cleaned_transformed.to_csv(os.path.join(data_cleaning_path, 'loan_data_2007_2014_cleaned_transformed.csv'))

# df_cleaned_transformed.to_csv('/Users/gan-m2/1-DataProcessing/data/1-data_cleaning_set/df_target_transformed_with_03_missing.csv')

# 3. Split data into two types (continuous and discrete)

In [23]:
def analyze_feature_type(df, threshold=0.05):
    """
    Analyze features in a dataframe to determine if they are likely discrete or continuous.
    
    :param df: Pandas DataFrame to analyze.
    :param threshold: Threshold for determining if a feature is discrete based on the ratio of unique values to total values.
    :return: A dictionary with feature names as keys and their likely type ('discrete' or 'continuous') as values.
    """
    feature_types = {}
    for column in df.columns:
        # Check the data type
        if df[column].dtype in ['int64', 'float64']:
            # Calculate the ratio of unique values to total values
            unique_ratio = df[column].nunique() / len(df)
            if unique_ratio < threshold:
                feature_types[column] = 'discrete'
            else:
                feature_types[column] = 'continuous'
        else:
            # Non-numeric types are typically discrete
            feature_types[column] = 'discrete'
    
    return feature_types

In [24]:
# Analyze the dataset
feature_column_list = df_cleaned_transformed.columns.tolist()
feature_analysis = analyze_feature_type(df_cleaned_transformed[feature_column_list])
discrete_features = [feature for feature, feature_type in feature_analysis.items() if feature_type == 'discrete']
continuous_features = [feature for feature, feature_type in feature_analysis.items() if feature_type == 'continuous']
# continuous_features = continuous_features.append('target')

In [25]:
# add the target column to the continuous_features list
continuous_features = continuous_features + ['target']
continuous_features

['installment',
 'annual_inc',
 'revol_bal',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'last_pymnt_amnt',
 'tot_cur_bal',
 'target']

In [26]:
# save both discrete and continuous features into dataframe and csv files with target column

df_discrete_features = df_cleaned_transformed[discrete_features]
df_continuous_features = df_cleaned_transformed[continuous_features]

df_discrete_features.to_csv(os.path.join(data_cleaning_path, 'df_discrete_features.csv'))
df_continuous_features.to_csv(os.path.join(data_cleaning_path, 'df_continuous_features.csv'))
